# Team Assignment - Intentionally Blank

The following libraries should be installed before proceeding further:
- pandas
- numpy
- matplotlib
- **folium** (conda install -c conda-forge folium)
- **branca** (Should automatically be installed with folium)
- **plotly** (conda install -c plotly plotly=5.10.0)

In [1]:
#Import all necessary libraries

import pandas as pd
import numpy as np
import math
from datetime import datetime
import seaborn as sns

import matplotlib.pyplot as plt

# also import these "new" libraries 
# Note: you may have to download an add them to your environment (using e.g. 'conda install -c conda-forge folium')
# !important! Install this version of plotly=5.10.0 or else some maps and animations may not render correctly
import plotly.express as px
import folium
from folium import plugins
from folium.plugins import HeatMap
from branca.element import Figure

In addition to the available datasets from the team assignment the original dataset is read for additional information about each ride. The additional information contains the geographical coordinates for the start position and the end position of each ride.

In [2]:
# Reading in weather and philadelphia_2017 data
df_weather = pd.read_csv("data/weather_hourly_philadelphia.csv")
df_philadelphia_2017 = pd.read_csv("data/philadelphia_2017.csv")

# Additionally read the philadelphia data from the official website
data_philadelphia_2017_Q1 = pd.read_csv("data/indego_Q1_2017.csv")
data_philadelphia_2017_Q2 = pd.read_csv("data/indego_Q2_2017.csv")
data_philadelphia_2017_Q3 = pd.read_csv("data/indego_Q3_2017.csv")
data_philadelphia_2017_Q4 = pd.read_csv("data/indego_Q4_2017.csv")

## Preparing and cleaning data

### Preparing and cleaning the philadelphia datasets

In [3]:
# Concatenate all quarters of philadelphia data and convert them to a dataframe
df_philadelphia_website = pd.concat([data_philadelphia_2017_Q1, data_philadelphia_2017_Q2, data_philadelphia_2017_Q3, data_philadelphia_2017_Q4], ignore_index=True)

In [ ]:
# Dropping the columns duration, start_station_id, end_station_id, plan_duration, trip_route_category, passholder_type, start_station, end_station
df_philadelphia_website.drop(['duration', 'start_station_id', 'end_station_id', 'plan_duration', 'trip_route_category', 'passholder_type', 'start_station', 'end_station', 'trip_id'], axis=1, inplace=True)
df_philadelphia_website

#Changing the type of start_time, end_time (df_philadelphia_2017 and df_philadelphia_website) and date_time (df_weather) to datetime type from pandas
df_philadelphia_website.loc[:,'start_time'] = pd.to_datetime(df_philadelphia_website['start_time'])
df_philadelphia_website.loc[:,'end_time'] = pd.to_datetime(df_philadelphia_website['end_time'])

df_philadelphia_2017.loc[:,'start_time'] = pd.to_datetime(df_philadelphia_2017['start_time'])
df_philadelphia_2017.loc[:,"end_time"] = pd.to_datetime(df_philadelphia_2017["end_time"])

#Sort dataframes by their corresponding time column
df_philadelphia_website.sort_values(["start_time"], inplace = True)
df_philadelphia_2017.sort_values(["start_time"], inplace = True)


#Resetting the indexes
df_philadelphia_website.reset_index(drop = True, inplace = True)
df_philadelphia_2017.reset_index(drop = True, inplace = True)

Before we further clean the dataset for philadelphia its important to mention that we first deleted the irrelevant stations because some duplicate rows happen to have the same start time, end time and bike ids but different start and end stations like the "Virtual Station". Therefore we deleted those stations first to avoid deleting useful records. In the following example you can see that deleting the duplicates first could lead to deleting useful records and keeping irrelevant records which are deleted either way later.

In [ ]:
df_philadelphia_2017[df_philadelphia_2017.duplicated(subset=['start_time', 'end_time', 'bike_id'], keep=False)].head(6)

The "3083" station does not exist as a real station. Furthermore the "Virtual Station" is used by staff to check in or check out a bike remotely for an event or when the bike was not properly checked in or out. Therefore we check the number of rows where one of those stations occur and delete those rows afterwards.

For further information please check the official site about the data and the station table [here](https://www.rideindego.com/about/data/).

In [ ]:
print(r'Number of rows in df_philadelphia_2017 with "Virtual Station" and "3083" in the start station or end station: ',
 df_philadelphia_2017[(df_philadelphia_2017["start_station_name"] == "Virtual Station") | (df_philadelphia_2017["end_station_name"] == "Virtual Station") |
  (df_philadelphia_2017["start_station_name"] == "3083") | (df_philadelphia_2017["end_station_name"] == "3083")
  ].shape[0])

In [ ]:
# Deleting "Virtual Station" and "3083" from the start_station_name and end_station_name columns
df_philadelphia_2017 = df_philadelphia_2017[(df_philadelphia_2017["start_station_name"] != "3083") & (df_philadelphia_2017["end_station_name"] != "3083") ]
df_philadelphia_2017 = df_philadelphia_2017[(df_philadelphia_2017["start_station_name"] != "Virtual Station") & (df_philadelphia_2017["end_station_name"] != "Virtual Station") ]

display(df_philadelphia_2017[(df_philadelphia_2017["start_station_name"] == "Virtual Station") | (df_philadelphia_2017["end_station_name"] == "Virtual Station") | (df_philadelphia_2017["start_station_name"] == "3083") | (df_philadelphia_2017["end_station_name"] == "3083")])

The "bike_id" should be unique to one bike. No trip with the same bike_id should start and end on the same time interval. 

In [ ]:
#Checking for any duplicates in philadelphia data
print("Number of duplicates in df_philadelphia_2017: ", df_philadelphia_2017.duplicated(subset=['start_time', 'end_time', 'bike_id']).sum())
print("Number of duplicates in df_philadelphia_website: ", df_philadelphia_website.duplicated(subset=['start_time', 'end_time', 'bike_id']).sum())

In [ ]:
#Dropping all duplicates for the philadelphia data depending on the start_time, end_time and bike_id
df_philadelphia_2017.drop_duplicates(subset=["start_time", "end_time", "bike_id"], keep="first", inplace=True)
df_philadelphia_website.drop_duplicates(subset=["start_time", "end_time", "bike_id"], keep="first", inplace=True)

display(df_philadelphia_2017[df_philadelphia_2017.duplicated(subset=['start_time', 'end_time', 'bike_id'])].head(10))
display(df_philadelphia_website[df_philadelphia_website.duplicated(subset=['start_time', 'end_time', 'bike_id'])].head(10))

In [ ]:
# Joining df_philadelphia_2017 with df_philadelphia_full on start_time, end_time, bike_id and dropping duplicate columns
df_philadelphia_2017_joined = df_philadelphia_2017.merge(df_philadelphia_website, on=["start_time", "end_time", "bike_id"], how="left")

df_philadelphia_2017_joined

In [ ]:
#Checking for missing values in philadelphia data
display(df_philadelphia_2017_joined[df_philadelphia_2017_joined.isnull().any(axis = 1)])

In [ ]:
df_philadelphia_2017_joined.dropna(inplace = True)	

In [ ]:
#Adding hour, month, weekday and trip duration(based on start and end time) to the table
df_philadelphia_2017_joined.loc[:,"trip_duration"] = df_philadelphia_2017_joined["end_time"] - df_philadelphia_2017_joined["start_time"] 
df_philadelphia_2017_joined.loc[:,"date"] = df_philadelphia_2017_joined["start_time"].dt.date
df_philadelphia_2017_joined.loc[:,"month"] = df_philadelphia_2017_joined["start_time"].dt.month
df_philadelphia_2017_joined.loc[:,"week"] = df_philadelphia_2017_joined["start_time"].dt.isocalendar().week
df_philadelphia_2017_joined.loc[:,"weekday"] = df_philadelphia_2017_joined["start_time"].dt.weekday
df_philadelphia_2017_joined.loc[:,"day"] = df_philadelphia_2017_joined["start_time"].dt.day
df_philadelphia_2017_joined.loc[:,"hour"] = df_philadelphia_2017_joined["start_time"].dt.hour
df_philadelphia_2017_joined

We also remove trips which are shorter than one minute and longer than one day because the bike sharing service does not allow those trips normally.

In [ ]:
df_philadelphia_2017_joined[df_philadelphia_2017_joined["trip_duration"] < "0 days 00:01:00"]

In [ ]:
df_philadelphia_2017_joined = df_philadelphia_2017_joined[df_philadelphia_2017_joined["trip_duration"] >= "0 days 00:01:00"]
df_philadelphia_2017_joined[df_philadelphia_2017_joined["trip_duration"] < "0 days 00:01:00"]

In [ ]:
df_philadelphia_2017_joined[df_philadelphia_2017_joined["trip_duration"] > "1 days"]

In [ ]:
df_philadelphia_2017_joined = df_philadelphia_2017_joined[df_philadelphia_2017_joined["trip_duration"] <= "1 days"]
df_philadelphia_2017_joined[df_philadelphia_2017_joined["trip_duration"] > "1 days"]

In [ ]:
df_philadelphia_cleaned = df_philadelphia_2017_joined.copy()
df_philadelphia_cleaned

### Preparing and cleaning the weather dataset

After preparing and cleaning the philadelphia dataset we need to do most of the procedure on the weather dataset.

In [ ]:
#Changing the type of date_time (df_weather) to datetime type from pandas
df_weather.loc[:, "date_time"] = pd.to_datetime(df_weather["date_time"])

#Sort the dataframe by their corresponding time column
df_weather.sort_values(["date_time"], inplace = True)

#Reset the index
df_weather.reset_index(drop = True, inplace =True)

First of all we check for duplicate rows in the weather data and delete them if they occur because we do not need multiple rows multiple times.

In [ ]:
#Checking for any duplicates in weather data
print("Number of duplicates in df_weather: ", df_weather.duplicated().sum())

In [ ]:
#Dropping all duplicates for the weather data
df_weather.drop_duplicates(subset= ["date_time"],inplace = True)

display(df_weather[df_weather.duplicated()])

We also check for null values in the weather data and delete them. It is important to note that we later add additional rows with null values but with the missing time intervals because we can interpolate them with the interpolate function in pandas.

In [ ]:
#Checking for missing values in weather data
display(df_weather[df_weather.isnull().any(axis = 1)])

In [ ]:
df_weather.dropna(inplace = True)

Since the weather data consists of data from the beginning of 2015 until the end of 2019, we want to have as well only the weather data for 2017.

In [ ]:
#Selecting data from the beginning of 2017 till the end of 2017
df_weather_2017 = df_weather[(df_weather["date_time"]>= "2017-01-01 00:00:00") & (df_weather["date_time"]< "2018-01-01 00:00:00" )]

In [ ]:
#Add missing intervals in the weather data
df_weather_2017.set_index("date_time", inplace = True)
df_weather_2017 = df_weather_2017.resample("H").asfreq()

The minimum temperature and maximum temperature will be interpolated linearly. The precipitation value is interpolated by taking one of the nearest existing values.

In [ ]:
# Interpolate missing values in the weather data
df_weather_2017["min_temp"].interpolate(method = "linear", inplace = True)
df_weather_2017["max_temp"].interpolate(method = "linear", inplace = True)
df_weather_2017["precip"].interpolate(method = "pad", inplace = True)
df_weather_2017.reset_index(drop=False, inplace = True)

# Check if null values in the weather data still exist
display(df_weather_2017[df_weather_2017.isnull().any(axis = 1)])
display(df_weather_2017)

All dataframes for reference:
* **Philadelphia data**
   * *df_philadelphia_cleaned*
* **Weather data**
   * *df_weather_2017*

## Descriptive Analytics

### Temporal Demand Patterns and Seasonality

#### Analysis of Rental during the day

First count the started rentals for each hour of the day for the whole dataset and vizualize it in a histogram.

In [ ]:
#Calculate the number of rentals in each hour (accumulated over the year)
df = df_philadelphia_cleaned
df.groupby(["hour"])["hour"].count().reset_index(name="n_rentals_within_hour")

In [ ]:
#Calculate the mean
mean = df.groupby(["hour"])["hour"].count().mean()
mean

In [ ]:
plt.hist(df["hour"], bins=range(0,25,1), color="green")
plt.axhline(mean , color = 'r', linestyle = '--')
plt.xlabel("Hour")
plt.ylabel("Number of rentals started")
plt.title("Development of started rentals during the day")
plt.xticks(range(0,25,2))
plt.grid(True)
min_xlim, max_xlim = plt.xlim()
plt.text(min_xlim*(-0.2), mean*1.1, 'Mean: {:.2f}'.format(mean))
plt.show()

To get average values we calculate the number of days for which we have data in our dataset and calculate the avarage number of rentals started within each hour in a day.

IMPORTANT: The average values only provide meaningful results if we assume that we have all meaningful rental transactions that actually occurred included in our data set or at least equally distributed missing transaction for each hour.

In [ ]:
#Check the number of days for which we have data in our set
number_days = len(df["date"].unique())
number_days

In [ ]:
#Calculate average values for each hour
df_avg_per_hour = df.groupby(["hour"])["hour"].count().divide(number_days).reset_index(name="avg_n_rentals_within_hour")
df_avg_per_hour

In [ ]:
#Calculate mean
mean= df_avg_per_hour["avg_n_rentals_within_hour"].mean()
mean

In [ ]:
plt.bar(
        df_avg_per_hour["hour"], 
        df_avg_per_hour["avg_n_rentals_within_hour"], 
        color="green"
)
plt.axhline(mean , color = 'r', linestyle = '--')
plt.xlabel("Hour")
plt.ylabel("Average number of rentals started within hour")
plt.title("Development of avg started rentals during the day")
min_xlim, max_xlim = plt.xlim()
plt.text(min_xlim*(-0.2), mean*1.1, 'Mean: {:.2f}'.format(mean))
plt.grid(True)
plt.show()

In the plots above we only considered the starting times of the rentals. To also considere the endtime, we calculate how many bikes in the respective hours were IN USE accumulated over the year. (e.g. rental from 0 o'clock to 2:45 would count as a usage in hour 0, 1 and 2). In the process of calculation we also have to considere that bike rentals can range over two days.

In [ ]:
#Calculate average number of bikes in use
df["end_time_hour"] = df["end_time"].dt.hour
df["end_time_date"] = df["end_time"].dt.date
df_values = pd.DataFrame()
for i in range(0,24,1):
    df_values[f"{i}"]= (((df["hour"] <= i) & (df["end_time_hour"] >= i)) 
                        | ((df["end_time_hour"] >= i) & (df["date"] < df["end_time_date"])))

df_sum = df_values.apply(lambda x: x.sum()/number_days).reset_index(name="avg_n_of_bikes_in_use").rename(columns={"index": "hour"})
df_sum

In [ ]:
#Calculate mean
mean = df_sum["avg_n_of_bikes_in_use"].mean()
mean

In [ ]:
plt.bar(df_sum["hour"], df_sum["avg_n_of_bikes_in_use"], color="green")
plt.axhline(mean , color = 'r', linestyle = '--')
plt.xlabel("Hour")
plt.ylabel("Average Number of Bikes in Use")
plt.title("Bikes in Use by Hours of the Day")
min_xlim, max_xlim = plt.xlim()
plt.text(min_xlim*(-0.2), mean*1.1, 'Mean: {:.2f}'.format(mean))
plt.grid(True)
plt.show()

##### Analysis and Interpretation of the Results:

The plots above all show that we have two peaks of bike rentals in the day. One peak demand at 8 am and the other peak at 5 pm. This peak can be a result of the rush-hour traffic. At these times most people are on their way to work/school or on their way back home. The demand around these times ( 8 am and 5 pm ) are also above average. Besides that, it also becomes clear that the bikes are mostly used during the day. During the night the demand is significantly below average (from 9 pm until 6 am).

#### Analysis of Rental during the week

First count the started rentals for each weekday of the day for the whole dataset and vizualize it in a histogram.

In [ ]:
#Calculate accumulated rentals per weekday
df.groupby(["weekday"])["weekday"].count().reset_index(name="n_rentals_within_weekday")

The mapping: {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thursday", 4: "Friday",5: "Saturday",6: "Sunday"}

In [ ]:
#Calculate mean
mean = df.groupby(["weekday"])["weekday"].count().mean()
mean

In [ ]:
plt.hist(df["weekday"], bins=range(0,8,1), color="green")
plt.axhline(mean , color = 'r', linestyle = '--')
plt.xlabel("Weekday")
plt.ylabel("Total Number of Rentals Started on Weekday")
plt.title("Development of Started Rentals During the Week")
plt.xticks(range(0,8,1))
min_xlim, max_xlim = plt.xlim()
plt.text(min_xlim*(-0.1), mean*1.1, 'Mean: {:.2f}'.format(mean))
plt.grid(True)
plt.show()

To get average values we calculate the number of weeks for which we have data in our dataset and calculate the avarage number of rentals started within each weekday.

IMPORTANT ASSUMPTION: The average values only provide meaningful results if we assume that we have all rental transactions that actually occurred included in our data set or at least equally distributed missing data points for each weekday. Otherwise the average values would provide misleading information. If we for example would have missing rental data for a specific weekday for several weeks, dividing by the total number of weeks would result to false average value for this weekday!

In [ ]:
#Check number of weeks we have in our set
number_weeks = len(df["week"].unique())
number_weeks

In [ ]:
#Calculate average values per weekday
df_average_per_weekday = df.groupby(["weekday"])["weekday"].count().divide(number_weeks).reset_index(name="avg_number_started_rentals")
df_average_per_weekday

In [ ]:
#Calculate mean
mean = df_average_per_weekday["avg_number_started_rentals"].mean()
mean

In [ ]:
plt.bar(
        df_average_per_weekday["weekday"], 
        df_average_per_weekday["avg_number_started_rentals"], 
        color="green"
)
plt.axhline(mean, color="r", linestyle="--")
plt.xlabel("Weekday")
plt.ylabel("Average Number of Rentals Started per Weekday")
plt.title("Development of Started Rentals During the Week")
min_xlim, max_xlim = plt.xlim()
plt.text(min_xlim*(-0.1), mean*1.1, 'Mean: {:.2f}'.format(mean))
plt.grid(True)
plt.show()

##### Analysis and Interpretation of Results:

The Bike rental demand is the lowest at the weekends (Saturdays and Sundays). The demand these days is significantly below average. The demand on Mondays is slightly below the average. We have a peak demand on Wednesdays. This development can possibly be rooted in the fact that most people don't work/ go to school or university on the weekends. This could mean that a big share of people who use the bike rental service are people who are on their way to work, school, or university.

#### Analysis of Rental during the Year

First count the started rentals for each month of the year for the whole dataset and vizualize it in a histogram.
Average values can not be calculated since we only have data of one year.

In [ ]:
#Calculate number of rentals per month
df.groupby(["month"])["month"].count().reset_index(name="n_of_retals_in_month")

In [ ]:
#Calculate mean
mean = df.groupby(["month"])["month"].count().mean()
mean

In [ ]:
plt.hist(df["month"], bins=range(0,14,1), color="green")
plt.axhline(mean, color="r", linestyle="--")
plt.xlabel("Month")
plt.ylabel("Total Number of Rentals Started in Month")
plt.title("Development of Started Rentals during Month")
plt.xticks(range(1,13,1))
min_xlim, max_xlim = plt.xlim()
plt.text(min_xlim*(-0.1), mean*(1.06), "Mean: {:.2f}".format(mean))
plt.grid(True)
plt.show()

In [ ]:
df_weather = df_weather_2017
df_weather["date"] = df_weather["date_time"].dt.date
df_weather["hour"] = df_weather["date_time"].dt.hour

#Join weather data with rental data and calculate average temperatures
df_merge = pd.merge(df, df_weather, how="left", left_on=["date", "hour"], right_on=["date", "hour"])
df_merge["avg_temp"] = df_merge[["max_temp", "min_temp"]].mean(axis="columns")

In [ ]:
#Calculate average temperatures for each month
df_temp_month = df_merge.groupby("month")["avg_temp"].mean().reset_index(name="avg_temp")
df_temp_month

In [ ]:
#Plot number of rentals together with average temperature
#Red dots would be the average temperature for each month
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
df_months = df.groupby(["month"])["month"].count().reset_index(name="n_of_retals_in_month")
ax1.bar(df_months["month"], df_months["n_of_retals_in_month"], color="green")
ax1.set_ylabel("Total Number of Rentals Started in Month")
plt.title("Development of Started Rentals and Temperature")
plt.xticks(range(1,13,1))
ax2.scatter(df_temp_month["month"], df_temp_month["avg_temp"], color="red")
ax2.set_ylabel("Average Temperature")
ax1.set_xlabel("Month")
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(df_temp_month["avg_temp"], df_months["n_of_retals_in_month"], color="red")
plt.xlabel("Average Temperature")
plt.ylabel("Bike Rental Demand")
plt.title("Development of Bike Rental Demand dependant on Average Temperature")
plt.grid(True)
plt.show()

##### Analysis and Interpretation of the Results:

The development of bike rental demand has similar development as the development of the average temperature during the year. 
The only difference can be seen at the peak. The average temperature has its peak in July whereas the bike rental demand has its peak in August. However, the difference in the bike rental demand in July and August is not that high. Another effect that could have an influence on the bike rental demand is the number of tourists. According to the annual report of Philadelphia, the number of tourists was the highest for the second and third quarters of the year (April to September). In these months the bike rental demand was always very close (April) or above the overall average of the bike rental demand. 
Especially for the month of December, January, February, and March the bike rental demand is clearly below the average.

Development of the tourist numbers according to the annual report: 
- Q1: 7 million
- Q2: 13.1 million
- Q3: 13.1 million
- Q4: 10.1 million

### Geographical Demand Patterns ##

In this section, we look at which stations in Philadelphia are particularly popular and which are not due to the given dataset.

First step: reading data from the bikeshare dataset from Ride Indego Philadelphia.

In [ ]:
df = df_philadelphia_cleaned.copy()

In [ ]:
data = df[["start_station_id", "start_station_name", "end_station_id", "end_station_name"]]
data

In [ ]:
data = df[["start_station_name", "end_station_name"]]
data

In the following we look at the stations where the ride was started and sort them in descending order of the number of rentals.

In [ ]:
df.groupby(["start_station_name"])["start_station_name"].count().reset_index(name = "rentals_per_station").sort_values(by=['rentals_per_station'], ascending=False)

Displaying only the five most popular start stations:

In [ ]:
df_most_popular = df.groupby(["start_station_name"])["start_station_name"].count().reset_index(name = "rentals_per_station").sort_values(by=['rentals_per_station'], ascending=False).head()
df_most_popular

In [ ]:
fig = plt.figure(figsize=(9,4))
plt.bar(df_most_popular["start_station_name"], df_most_popular["rentals_per_station"])
plt.rc('xtick', labelsize = 10)
plt.rc('ytick', labelsize = 10)
plt.xticks(rotation=45, ha='right')
plt.xlabel("name of the station", size = 'medium')
plt.ylabel("number of rentals per station", size = 'medium')
plt.title("Top 5 most popular start stations", size = 'x-large')
plt.show()

Displaying only the five least popular start stations:

In [ ]:
df_least_popular = df.groupby(["start_station_name"])["start_station_name"].count().reset_index(name = "rentals_per_station").sort_values(by=['rentals_per_station'], ascending=False).tail()
df_least_popular

In [ ]:
fig = plt.figure(figsize=(9,4))
plt.bar(df_least_popular["start_station_name"], df_least_popular["rentals_per_station"])
plt.rc('xtick', labelsize = 10)
plt.rc('ytick', labelsize = 10)
plt.xticks(rotation=45, ha='right')
plt.xlabel("name of the station", size = 'medium')
plt.ylabel("number of rentals per station", size = 'medium')
plt.title("Top 5 least popular start stations", size = 'x-large')
plt.show()

In the following we look at the stations where the ride was terminated and sort them in descending order of the number of rentals.

In [ ]:
df.groupby(["end_station_name"])["end_station_name"].count().reset_index(name = "rentals_per_station").sort_values(by=['rentals_per_station'], ascending=False)

Displaying only the five most popular end stations:

In [ ]:
df_most_popular_end = df.groupby(["end_station_name"])["end_station_name"].count().reset_index(name = "rentals_per_station").sort_values(by=['rentals_per_station'], ascending=False).head()
df_most_popular_end

In [ ]:
fig = plt.figure(figsize=(9,4))
plt.bar(df_most_popular_end["end_station_name"], df_most_popular_end["rentals_per_station"])
plt.rc('xtick', labelsize = 10)
plt.rc('ytick', labelsize = 10)
plt.xticks(rotation=45, ha='right')
plt.xlabel("name of the station", size = 'medium')
plt.ylabel("number of rentals per station", size = 'medium')
plt.title("Top 5 most popular end stations", size = 'x-large')
plt.show()

Displaying only the five least popular end stations:

In [ ]:
df_least_popular_end = df.groupby(["end_station_name"])["end_station_name"].count().reset_index(name = "rentals_per_station").sort_values(by=['rentals_per_station'], ascending=False).tail()
df_least_popular_end

In [ ]:
df_least_popular_end = df_least_popular_end.replace(['39th & Mt. Vernon, Mantua Haverford Community Center', '40th & Baltimore, Trolley Portal'], ['39th & Mt. Vernon', '40th & Baltimore'])
print(df_least_popular_end)

In [ ]:
fig = plt.figure(figsize=(9,4))
plt.bar(df_least_popular_end["end_station_name"], df_least_popular_end["rentals_per_station"])
plt.rc('xtick', labelsize = 10)
plt.rc('ytick', labelsize = 10)
plt.xticks(rotation=45, ha='right')
plt.xlabel("name of the station", size = 'medium')
plt.ylabel("number of rentals per station", size = 'medium')
plt.title("Top 5 least popular end stations", size = 'x-large')
plt.show()

#### Analysis of the results:

After looking at the top 5 most popular and top 5 least popular start and end stations, it is especially noticeable that the five most popular start stations are also the five most popular end stations in Philadelphia. Apparently, there is a high demand for renting bikes between these stations. According to Google Maps these stations are located in the center of the city and are no more than 1.6 miles apart from each other. This shows, that the bicycles are mainly used for short distances in the city center.

### Key Performance Indicators (KPIs)

#### Coverage

In the next part we use the geographical coordinates of the original dataset from the indego website.

First of all we create a dataframe where we group the rows by the start station and the corresponding coordinates. Furthermore we aggregate the rows into a new column "counts".

In [ ]:
phil_groupedby_start = df_philadelphia_cleaned.groupby([ 'start_station_name', 'start_lat', 'start_lon']).size().reset_index(name='counts')

phil_groupedby_start

We plot the number of trips per start station. For visualizing which stations stations have the most demand. For this we use circle markers to visualize each station. Bigger circle markers indicate more rented bikes for the corresponding station and clicking on a specific marker reveals additional information like the specific trip count and the station name.

In [ ]:
figure_start_station = Figure(width=950,height=900) 
map_start_station = folium.Map(dragging=True,zoom_control=True, control_scale=True, max_zoom=18, min_zoom=13,scrollWheelZoom=False,doubleClickZoom=False, width=900,height=800)
for index, row in phil_groupedby_start.iterrows():
    iframe = folium.IFrame(f"Station name: {row['start_station_name']} <br> Trips per station :  {row['counts']}", width=250, height=70)
    popup = folium.Popup(iframe)
    folium.Circle([row['start_lat'], row['start_lon']], radius=math.sqrt(row['counts']), color='red', fill_color='orange', fill_opacity=0.3, popup=popup).add_to(map_start_station)
south_west_b = phil_groupedby_start[['start_lat', 'start_lon']].min().values.tolist()
north_east_b = phil_groupedby_start[['start_lat', 'start_lon']].max().values.tolist()
map_start_station.fit_bounds([south_west_b, north_east_b])
figure_start_station.add_child(map_start_station)
title = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format("Number of trips per start station")   
map_start_station.get_root().html.add_child(folium.Element(title))
figure_start_station

As we can see most bikes are rented in stations near the center of philadelphia like "18th & JFK", "Rittenhouse Square" and "15th & Market" but also near main roads like the stations "15th & Spruce", "Rodin Museum" and "23rd & South". More possibilities why some stations could be more used are hotspots like Museums for example the "Rodin Museum" or the "Philadelphia Museum of Art".

Now we do the same procedure for the end stations, first grouping them like in the previous steps.

In [ ]:
phil_groupedby_end = df_philadelphia_cleaned.groupby([ 'end_station_name', 'end_lat', 'end_lon']).size().reset_index(name='counts')

phil_groupedby_end

In [ ]:
figure_end_station = Figure(width=950,height=900) 
map_end_station = folium.Map(dragging=True,zoom_control=True, control_scale=True, max_zoom=18, min_zoom=13,scrollWheelZoom=False,doubleClickZoom=False, width=900,height=800)
for index, row in phil_groupedby_end.iterrows():
    iframe = folium.IFrame(f"Station name: {row['end_station_name']} <br> Trips per station :  {row['counts']}", width=250, height=70)
    popup = folium.Popup(iframe)
    folium.Circle([row['end_lat'], row['end_lon']], radius=math.sqrt(row['counts']), color='red', fill_color='orange', fill_opacity=0.3, popup=popup).add_to(map_end_station)
south_west_b = phil_groupedby_end[['end_lat', 'end_lon']].min().values.tolist()
north_east_b = phil_groupedby_end[['end_lat', 'end_lon']].max().values.tolist()
map_end_station.fit_bounds([south_west_b, north_east_b])
figure_end_station.add_child(map_end_station)
title = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format("Trips per end station")   
map_end_station.get_root().html.add_child(folium.Element(title))
figure_end_station

As we can see the overall pattern stays the same. In the center of the city are the more popular stations and going further away from the center are the least popular stations. The least popular stations could be combined with other stations near it to save maintainance costs spacing the stations out the farther they are away from the center of philadelphia.

It is also interesting to see how much the stations cover the area of philadelphia. Therefore we loaded Geojson data of the philadelphia neighborhoods and their borders to visualize which parts of philadelphia are covered by the stations.

In [ ]:
figure_end_station = Figure(width=950,height=900) 
map_with_geoJson = folium.Map(location=[39.96,-75.18],dragging=True,zoom_start=12,zoom_control=True, control_scale=True, max_zoom=18, min_zoom=12,scrollWheelZoom=False,doubleClickZoom=False, width=900,height=800)
folium.GeoJson(data="data/Neighborhoods_Philadelphia.geojson").add_to(map_with_geoJson)

for index, row in phil_groupedby_end.iterrows():
    iframe = folium.IFrame(f"Station name: {row['end_station_name']} <br> Trips per station :  {row['counts']}", width=250, height=70)
    popup = folium.Popup(iframe)
    folium.Circle([row['end_lat'], row['end_lon']], radius=math.sqrt(row['counts']), color='red', fill_color='orange', fill_opacity=0.3, popup=popup).add_to(map_with_geoJson)

south_west_b = phil_groupedby_end[['end_lat', 'end_lon']].min().values.tolist()
north_east_b = phil_groupedby_end[['end_lat', 'end_lon']].max().values.tolist()
figure_end_station.add_child(map_with_geoJson)
title = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format("Trips per end station with philadelphia neighborhood")   
map_with_geoJson.get_root().html.add_child(folium.Element(title))
figure_end_station

As we can see, mostly the center is covered by the stations and the part of south philadelphia nearest to the center. This makes sense because the center has the most demand but if the demand rises with each year it could be worth adding stations near other hotspots and farther from the center.

Furthermore we can plot the demand per hour for the start station and end stations.

In [ ]:
grouped_by_start_station_hour = df_philadelphia_cleaned.groupby(['start_station_name', 'start_lat', 'start_lon', 'hour']).size().reset_index(name='counts')
fig = px.scatter_mapbox(grouped_by_start_station_hour, lat="start_lat", lon="start_lon", animation_frame="hour", 
                        size= "counts",size_max= 28,opacity=0.8, animation_group='start_station_name',
                        zoom=12.3, height=1000, width=900, hover_name="start_station_name",
                        hover_data=["start_lat", "start_lon"], title='Trips per start station for each hour')
fig.update_geos(center=dict(lon=-75.14, lat=39.96))
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    font=dict(
            size=18
        ))
fig

As we can see between 8:00 AM and 9:00 AM the majority of trips start outside of the center. One reason could be that the majority rents a bike to go to work or that most people live outside of the center. It is also interesting to mention that at 17:00 PM the majority of the trips start in the center or near the center. This also supports the assumption that many people use the bikes for going to work and back home from work because 17:00 PM is around the time where work ends for most people. Further research in comparing work hours and the geographical patterns could lead to more insights.

In [ ]:
grouped_by_end_station_hour = df_philadelphia_cleaned.groupby(['end_station_name', 'end_lat', 'end_lon', 'hour']).size().reset_index(name='counts')
fig = px.scatter_mapbox(grouped_by_end_station_hour, lat="end_lat", lon="end_lon", animation_frame="hour", 
                        size= "counts",size_max= 27,opacity=0.8, animation_group='end_station_name',
                        zoom=12.3, height=1000, width=900, hover_name="end_station_name",
                        hover_data=["end_lat", "end_lon"], title='Trips per end station for each hour')
fig.update_geos(center=dict(lon=-75.14, lat=39.96))
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    font=dict(
            size=18
        ))
fig

Look at the hourly trips per end station we can see this time that between 8:00 AM and 9:00 AM most trips end near the center instead of outside like for the start stations. Like wise for 17:00 PM less trips end in the center and more trips end outside of it.

In the following we also plotted the trips per start station and end station for each month. This could be useful for observing if new stations are used sufficiently.

In [ ]:
grouped_by_start_station_month = df_philadelphia_cleaned.groupby(['start_station_name', 'start_lat', 'start_lon', 'month']).size().reset_index(name='counts')
fig = px.scatter_mapbox(grouped_by_start_station_month, lat="start_lat", lon="start_lon", animation_frame="month", 
                        size= "counts",size_max= 25,opacity=0.8, animation_group='start_station_name',
                        zoom=12.3, height=1000, width=900, hover_name="start_station_name",
                        hover_data=["start_lat", "start_lon"], title='Trips per start station for each month')
fig.update_geos(center=dict(lon=-75.14, lat=39.96))
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    font=dict(
            size=18
        ))
fig

In [ ]:
grouped_by_end_station_month = df_philadelphia_cleaned.groupby(['end_station_name', 'end_lat', 'end_lon', 'month']).size().reset_index(name='counts')
fig = px.scatter_mapbox(grouped_by_end_station_month, lat="end_lat", lon="end_lon", animation_frame="month", 
                        size= "counts",size_max= 25,opacity=0.8, animation_group='end_station_name',
                        zoom=12.3, height=1000, width=900, hover_name="end_station_name",
                        hover_data=["end_lat", "end_lon"], title='Trips per end station for each month')
fig.update_geos(center=dict(lon=-75.14, lat=39.96))
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    font=dict(
            size=18
        ))
fig

We could problably get further insights on the utilizations and the coverage for each station like which stations are overloaded with bikes or which stations have no bikes available if the station data existed in hourly steps for 2017. With the live data on the [indego website](https://www.rideindego.com/about/data/) which contains all of these informations this could be possible if it existed for previous years.

#### Rental Durations

Rental durations are closely tied to revenue because the revenue model of the bike sharing company is based on the rental time/ duration thus its relevant to observe the rental duration to get information on the company's performance.

In [ ]:
#Calculate the sum of rental durations for each month
#Calculate the average rental duration per rental for each month
df = df_philadelphia_cleaned
df_duration_monthly = df.groupby("month")["trip_duration"].sum().reset_index(name="sum_duration_bike_rentals")
df_duration_monthly["duration_in_hours"] = df_duration_monthly["sum_duration_bike_rentals"].dt.total_seconds().divide(60*60)
df_duration_monthly["duration_in_minutes"] = df_duration_monthly["sum_duration_bike_rentals"].dt.total_seconds().divide(60)
df_duration_monthly["number_of_rentals"] = df.groupby("month")["month"].count().reset_index(drop=True)
df_duration_monthly["avg_duration_per_rental_in_min"] = df_duration_monthly["duration_in_minutes"] / df_duration_monthly["number_of_rentals"]
df_duration_monthly

In [ ]:
#Vizualize development of summed durations
plt.bar(df_duration_monthly["month"],
       df_duration_monthly["duration_in_hours"],
       color="green")
plt.grid(True)
plt.xticks(range(1,13,1))
plt.xlabel("Months")
plt.ylabel("Sum of Rental Durations")
plt.title("Development of the Summed Rental Durations")
plt.show()

In [ ]:
#Vizualize development of average durations for each month
plt.bar(df_duration_monthly["month"],
       df_duration_monthly["avg_duration_per_rental_in_min"],
       color="green")
plt.grid(True)
plt.xticks(range(1,13,1))
plt.xlabel("Months")
plt.ylabel("Avg Duration per Rental in min")
plt.title("Development of Average Rental Duration")
plt.show()

In [ ]:
#Calculate the sum of rental durations for each week
#Calculate the average rental duration per rental for each week
df_duration_weekly = df.groupby("week")["trip_duration"].sum().reset_index(name="sum_duration_bike_rentals")
df_duration_weekly["duration_in_hours"] = df_duration_weekly["sum_duration_bike_rentals"].dt.total_seconds().divide(60*60)
df_duration_weekly["duration_in_minutes"] = df_duration_weekly["sum_duration_bike_rentals"].dt.total_seconds().divide(60)
df_duration_weekly["number_of_rentals"] = df.groupby("week")["week"].count().reset_index(drop=True)
df_duration_weekly["avg_duration_per_rental_in_min"] = df_duration_weekly["duration_in_minutes"] / df_duration_weekly["number_of_rentals"]
df_duration_weekly.head()

In [ ]:
#Vizualize development of summed durations
plt.rcParams["figure.figsize"] = (20,8)
plt.plot(df_duration_weekly["week"],
       df_duration_weekly["duration_in_hours"],
       color="green")
plt.grid(True)
plt.xticks(range(1,53,1))
plt.xlabel("Weeks")
plt.ylabel("Sum of Rental Durations")
plt.title("Development of the Summed Rental Durations")
plt.show()

In [ ]:
#Vizualize development of average durations for each week
plt.plot(df_duration_weekly["week"],
       df_duration_weekly["avg_duration_per_rental_in_min"],
       color="green")
plt.grid(True)
plt.xticks(range(1,53,1))
plt.xlabel("Weeks")
plt.ylabel("Average of Rental Durations")
plt.title("Development of the Average Rental Durations")
plt.show()

##### Interpretation of the Results

- The summed rental durations have a similar development as the summed number of rentals 
- Our bikes are more used in the summer months especially from June to October
- This can be seen in the monthly as well as the weekly aggregation 
- The average rental duration per ride lies between 15 and 22 minutes with the average rental duration being especially high from April to July

##### Rental Duration Dependant on Weather

In [ ]:
df_weather = df_weather_2017
df_weather["date"] = df_weather["date_time"].dt.date
df_weather["hour"] = df_weather["date_time"].dt.hour

#Join weather data with rental data and calculate average temperatures
df_merge = pd.merge(df, df_weather, how="left", left_on=["date", "hour"], right_on=["date", "hour"])
df_merge["avg_temp"] = df_merge[["max_temp", "min_temp"]].mean(axis="columns")

In [ ]:
df_duration_daily = df_merge.groupby("date").agg({"trip_duration": lambda x: x.sum(), 
                                                  "precip": lambda x: x.sum(), 
                                                  "avg_temp": lambda x: x.mean()})
df_duration_daily["duration_in_hours"] = df_duration_daily["trip_duration"].dt.total_seconds().divide(60*60)
df_duration_daily["duration_in_minutes"] = df_duration_daily["trip_duration"].dt.total_seconds().divide(60)
df_duration_daily["number_of_rentals"] = df.groupby("date")["date"].count()
df_duration_daily["avg_duration_per_rental_in_min"] = df_duration_daily["duration_in_minutes"] / df_duration_daily["number_of_rentals"]
df_duration_daily_precip = df_duration_daily[df_duration_daily["precip"] > 0]
df_duration_daily_no_precip = df_duration_daily[df_duration_daily["precip"] == 0]
df_duration_daily

In [ ]:
#Calculate average summed rental duration per day
avg_summed_duration_precip = df_duration_daily_precip["duration_in_hours"].mean()
avg_summed_duration_no_precip = df_duration_daily_no_precip["duration_in_hours"].mean()
print(f"{avg_summed_duration_precip= }\n{avg_summed_duration_no_precip= }")

In [ ]:
#Calculate average rental duration per rental
avg_duration_per_rental_precip = df_duration_daily_precip["avg_duration_per_rental_in_min"].mean()
avg_duration_per_rental_no_precip = df_duration_daily_no_precip["avg_duration_per_rental_in_min"].mean()
print(f"{avg_duration_per_rental_precip= }\n{avg_duration_per_rental_no_precip= }")

In [ ]:
#Calculate average rental duration and average summed rental duration per day dependen on temperature
df_duration_daily["temp_bin"] = pd.cut(df_duration_daily.avg_temp, 10)
df_temp_bins = df_duration_daily.groupby("temp_bin").agg({"duration_in_hours": lambda x: x.mean(), 
                                          "avg_duration_per_rental_in_min": lambda x: x.mean()}).reset_index()
df_temp_bins["temp_bin"] = df_temp_bins["temp_bin"].astype(str)
df_temp_bins

In [ ]:
plt.rcParams["figure.figsize"] = (20,8)
plt.bar(df_temp_bins["temp_bin"], df_temp_bins["duration_in_hours"], color="green")
plt.xlabel("Temperature")
plt.ylabel("Average Summed Trip Durations per Day in Hours")
plt.title("Development of Average Summed Trip Durations per Day Dependent on Temperature")
plt.grid(True)
plt.show()

In [ ]:
plt.bar(df_temp_bins["temp_bin"], df_temp_bins["avg_duration_per_rental_in_min"], color="green")
plt.xlabel("Temperature")
plt.ylabel("Average Trip Duration in Minutes")
plt.title("Development of Average Trip Durations Dependent on Temperature")
plt.grid(True)
plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]
plt.show()

##### Interpretation of the Results

- When additionally taking weather data into account, we can observe that the average summed rental durations per day are lower on days with precipitation. This is also the case for the average rental duration per day
- Besides that, the average summed rental durations per day also increase with the increase of the temperature
- With the average rental duration per trip we only slightly see this effect (there is not a continuous increase and not a very extreme one as with summed rental durations)

#### Bike Utilization

Bike utilization gives us information on how much the different bikes are used.
This information is relevant to get information about possible unutilized /unused bikes which are maybe actually not needed (unused ressources/capacitiy).
Moreover we analyse how many unique bikes at which time and at which weather condition.

In [ ]:
#approximate how many individual bikes there are (we have no information about total number of bikes)
number_of_bikes = len(df["bike_id"].unique())
total_number_of_rentals = len(df) # each row is a rental
print(f"{number_of_bikes=}, {total_number_of_rentals=}")

In [ ]:
#Calculate total number of rides for each bike over the whole year
df_bike_ride_count = df.groupby("bike_id")["start_time"].count().reset_index(name="total_number_of_rides").sort_values(by="total_number_of_rides", ascending=False).reset_index(drop=True)
df_bike_ride_count["cumulative_ride_count"] = df_bike_ride_count["total_number_of_rides"].cumsum()
df_bike_ride_count["percentage_number_of_bikes"] = df_bike_ride_count.index.to_series().add(1).divide(number_of_bikes)
df_bike_ride_count["percentage_ride_count"] = df_bike_ride_count["cumulative_ride_count"].divide(total_number_of_rentals)
df_bike_ride_count

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)
plt.plot(df_bike_ride_count.index.to_series(), df_bike_ride_count["total_number_of_rides"], color="green")
plt.grid(True)
plt.ylabel("Total Number of Rides")
plt.xlabel("Bike Count")
plt.title("Total Number of Rides per Bike")
plt.show()

In [ ]:
plt.plot(df_bike_ride_count.index.to_series(), df_bike_ride_count["cumulative_ride_count"], color="green")
plt.grid(True)
plt.xlabel("Bike Count")
plt.ylabel("Number of Rentals")
plt.title("Cumulative Number of Rentals")
plt.show()

In [ ]:
plt.plot(df_bike_ride_count["percentage_number_of_bikes"], df_bike_ride_count["percentage_ride_count"], 
         color="green")
plt.grid(True)
plt.xlabel("Number of Bikes in Percent")
plt.ylabel("Number of Rentals in Percent")
plt.title("Number of Bikes and Rentals in Percent")
plt.show()

In [ ]:
#Utilization by month
df_util_month = df.groupby("month")["bike_id"].nunique().reset_index(name="number_of_bikes_used")
df_util_month["utilization"] = df_util_month["number_of_bikes_used"].divide(number_of_bikes)
df_util_month

In [ ]:
plt.bar(df_util_month["month"], df_util_month["number_of_bikes_used"], color="green")
plt.xlabel("Months")
plt.ylabel("Number of Used Bikes")
plt.title("Utilization of Bikes per Month")
plt.grid(True)
plt.show()

In [ ]:
plt.bar(df_util_month["month"], df_util_month["utilization"], color="green")
plt.xlabel("Months")
plt.ylabel("Utilization in Percent")
plt.title("Utilization of Bikes per Month")
plt.grid(True)
plt.show()

It seems that our initial assumption of the total number of bikes could be untrue for the different months.
It could be possible that bikes are added in some months. For example in Mai there is an sudden increase of the bike utilization. 
We will check it in the following steps.

In [ ]:
set(df[df["month"]==5]["bike_id"].unique()) - set(df[df["month"]==4]["bike_id"].unique())

 It seems that bikes above 11700 are added in Mai.

In [ ]:
#Check for bikes with BikeID above 11700 for months 1 to 4
for i in range(1,5):
    print((df[df["month"]==i]["bike_id"] > 11700).any())

In [ ]:
#Get the highest BikeID for each month
df.groupby("month")["bike_id"].max()

In [ ]:
#Weekly values
df_bike_count_weekly = df.groupby("week")["bike_id"].nunique().reset_index(name="number_of_bikes")

In [ ]:
plt.plot(df_bike_count_weekly["week"], df_bike_count_weekly["number_of_bikes"], color="green")
plt.xlabel("Week")
plt.ylabel("Number of Used Bikes")
plt.title("Number of Used Bikes per Week")
plt.xticks(range(0,53,2))
plt.grid(True)
plt.show()

In [ ]:
#Daily values
df_bike_count_daily = df.groupby(["date"])["bike_id"].nunique().reset_index(name="number_of_bikes")
df_bike_count_daily

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.plot(df_bike_count_daily["date"], df_bike_count_daily["number_of_bikes"], color="green")
plt.grid(True)
plt.yticks(range(0,1000,100))
plt.xlabel("Day")
plt.ylabel("Number of Bikes")
plt.title("Number of Bikes Used per Day")
plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]
plt.show()

In [ ]:
#Average bike utilization by weekday
df_bike_count_daily["date"] = pd.to_datetime(df_bike_count_daily["date"])
df_bike_count_daily["weekday"] = df_bike_count_daily["date"].dt.weekday
df_weekday = df_bike_count_daily.groupby("weekday")["number_of_bikes"].mean().reset_index(name="avg_n_bikes")
df_weekday

In [ ]:
plt.bar(df_weekday["weekday"], df_weekday["avg_n_bikes"], color="green")
plt.xlabel("Weekday")
plt.ylabel("Number of Bikes")
plt.title("Average Number of Bikes per Weekday")
plt.grid(True)
plt.show()

In [ ]:
#Average bike utilization by hour of day
df_bike_count_hourly = df.groupby(["date", "hour"])["bike_id"].nunique().reset_index(name="number_of_bikes")
df_hour_of_day = df_bike_count_hourly.groupby("hour")["number_of_bikes"].mean().reset_index(name="avg_n_bikes")
df_hour_of_day

In [ ]:
plt.bar(df_hour_of_day["hour"], df_hour_of_day["avg_n_bikes"], color="green")
plt.xlabel("Hour")
plt.ylabel("Number of Bikes")
plt.title("Average Number of Used Bikes per Hour")
plt.xticks(range(0,24,1))
plt.grid(True)
plt.show()

##### Interpretation of the Results

- We approximated which number of total bikes there could be in the fleet 
- We  found a total of 1249 individual bikes in the data set 
- However, there are likely bikes added over time, since some bikeID's (especially higher ones) only occur starting at a certain month in the year
- This fact has to be considered when analyzing the results

- When analyzing the utilization of the individual bikes we can observe that only a few bikes (about five) are used significantly more than the others. And about two hundred bikes are used less than most bikes. But for most bikes, as seen in the graph the utilization does not vary too much. Besides that, because some bikes are likely to be added over time, it makes sense that these bikes were not used as much as the rest of the bikes.

- The bike utilization per month (considering a total number of bikes of 1249) shows that it was likely that bikes were added for example in Mai. Since we can see a sudden increase in the utilization (in the months before the utilization was quite the same)

- When analyzing the average number of bikes used per weekday it becomes clear that there are on average more bikes used during the week than during the weekends. Similar to the development of the number of rides.
- The development during the day also shows similar results to the development of the number of rides. We have clear peaks at 8 am and 5 pm.

##### Bike Utilization Dependent on Weather

In [ ]:
#Calculate average number of bikes used on hour with precipitation vs. one with none
df_hourly = df_merge.groupby(["date", "hour"]).agg({"bike_id": lambda x: x.nunique(), 
                                                    "precip": lambda x: x.sum(),
                                                   "avg_temp": lambda x: x.mean()})
df_hourly = df_hourly.rename(columns={"bike_id": "n_bikes_used"})
df_no_precip_hourly = df_hourly[df_hourly["precip"] == 0]
df_precip_hourly = df_hourly[df_hourly["precip"] > 0]
print(f"with precip: {df_precip_hourly['n_bikes_used'].mean()} avg bikes used per hour\nwithout: {df_no_precip_hourly['n_bikes_used'].mean()} avg bikes used per hour")

In [ ]:
#Calculate average number of bikes used on hour dependent on temperature
df_hourly["temp_bin"] = pd.cut(df_hourly["avg_temp"], 10)
df_temp_bins_hourly = df_hourly.groupby("temp_bin")["n_bikes_used"].mean().reset_index(name="avg_n_bikes_per_hour")
df_temp_bins_hourly["temp_bin"] = df_temp_bins_hourly["temp_bin"].astype(str)
df_temp_bins_hourly

In [ ]:
plt.rcParams["figure.figsize"] = (20,8)
plt.bar(df_temp_bins_hourly["temp_bin"], df_temp_bins_hourly["avg_n_bikes_per_hour"], color="green")
plt.xlabel("Temperature")
plt.ylabel("Average Number of Bikes Used per Hour")
plt.title("Average Number of Bikes Used per Hour Dependant on Temperature")
plt.grid(True)
plt.show()

In [ ]:
#Calculate average number of bikes used on day with precipitation vs. one with none
df_daily = df_merge.groupby("date").agg({"bike_id": lambda x: x.nunique(), 
                                         "precip": lambda x: x.sum(),
                                         "avg_temp": lambda x: x.mean()})
df_daily = df_daily.rename(columns={"bike_id": "n_bikes_used"})
df_no_precip_daily = df_daily[df_daily["precip"] == 0]
df_precip_daily = df_daily[df_daily["precip"] > 0]
print(f"with precip: {df_precip_daily['n_bikes_used'].mean()} avg bikes used per day\nwithout: {df_no_precip_daily['n_bikes_used'].mean()} avg bikes used per day")

In [ ]:
#Calculate average number of bikes used on day dependent on temperature
df_daily["temp_bin"] = pd.cut(df_daily["avg_temp"], 10)
df_daily

In [ ]:
df_temp_bins = df_daily.groupby("temp_bin")["n_bikes_used"].mean().reset_index(name="avg_n_bikes_per_day")
df_temp_bins["temp_bin"] = df_temp_bins["temp_bin"].astype(str)
df_temp_bins

In [ ]:
plt.bar(df_temp_bins["temp_bin"], df_temp_bins["avg_n_bikes_per_day"], color="green")
plt.xlabel("Temperature")
plt.ylabel("Average Number of Bikes Used per Day")
plt.title("Average Number of Bikes Used per Day Dependant on Temperature")
plt.grid(True)
plt.show()

##### Interpretation of the Results

The average number of bikes used per hour is lower when precipitationis recorded. Besides that, we see a steady increase in the average number of bikes used per hour with the increase in the temperature. This is also the case for the average number of bikes used per day.

#### Revenue

For this KPI we will calculate the revenue for every trip based on the given user_type. We can not calculate the revenue per user,
since we do not have any information about specific users in our dataset. Beforehand an analysis on the user_type`s themselves was 
done, to gain a deeper understanding of those.

First we look into the total distribution of rides by user types visualized by a pie chart.

In [ ]:
# Count all rides per user type and sort descending
df_philadelphia_cleaned.groupby(["user_type"])["user_type"].count().reset_index(name = "n_rides").sort_values(by=['n_rides'], ascending=False)

In [ ]:
# Put data into Series to visualize as Pie Chart
slices = [662593, 99043, 6787, 6236, 5972, 1594]
user_types = ['Indego30','Walk-up', 'Indego365', 'OneDayPass', 'IndegoFlex', 'TwoDayPass']
cols = ['b','r','c','g', 'orange', 'magenta']

In [ ]:
# Create pie chart
fig = plt.figure(figsize=(15,15))
plt.pie(slices,
labels = user_types,
labeldistance = 1,
pctdistance = 0.9,
colors = cols,
startangle = 0,
shadow = True,
explode =(0.1,0,0,0,0,0),
autopct ='%1.1f%%',
textprops={'fontsize': 16})
plt.title('Total Number of Rides by User Type', fontsize=20)
plt.legend(fontsize=12)

# Print the chart
plt.show()

Now we will look at the trip duration for every user type.

In [ ]:
# Preparing new dataframe with only trip_duration (converted to minutes) and the user_type
df_phil_rentDuration = df_philadelphia_cleaned[['trip_duration', 'user_type']].copy(deep=True)
df_phil_rentDuration.loc[:, 'trip_duration'] = df_phil_rentDuration['trip_duration'].dt.seconds / 60
df_phil_rentDuration.rename(columns={'trip_duration': 'trip_duration in minutes'}, inplace=True)
df_phil_rentDuration['trip_duration in minutes'] = df_phil_rentDuration['trip_duration in minutes'].astype(int)
# The dataset from above needs to be resetted, so it doesnt need to be done here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
df_phil_rentDuration.reset_index(drop = True, inplace = True)

# Final dataframe
df_phil_rentDuration

In [ ]:
# Group by on the user_type to find out the trip_duration for every user_type
df_phil_rentDuration = df_phil_rentDuration.groupby(['user_type']).sum().reset_index()

# Every user_type devided by the total amount of trip_duration
df_phil_rentDuration['percentage of trip_duration in minutes'] = df_phil_rentDuration['trip_duration in minutes'] / df_phil_rentDuration['trip_duration in minutes'].sum()
df_phil_rentDuration

In [ ]:
# Plotting
fig = plt.figure(figsize=(12,7))
plt.bar(df_phil_rentDuration['user_type'], df_phil_rentDuration['percentage of trip_duration in minutes'])

# Styling the plot
plt.rc('xtick', labelsize = 10)
plt.rc('ytick', labelsize = 10)
plt.xlabel('Specific user types', size = 'medium')
plt.ylabel('Trip duration in minutes', size = 'medium')
plt.title('Trip duration of specific user types', size = 'x-large')
plt.show()

Next we take a look at monthly development of rides by user types to look for any seasonal patterns:

In [ ]:
# Create Dataframe grouped by User types and months
df_months_usertype = df_philadelphia_cleaned.groupby(['month','user_type'])["user_type"].count().reset_index(name = "n_rides").sort_values(by=['month'], ascending=True)

# Split Dataframe into multiple Dataframes for each usertype
df_months_indego30 = df_months_usertype[df_months_usertype['user_type'] == "Indego30"]
df_months_indego365 = df_months_usertype[df_months_usertype['user_type'] == "Indego365"]
df_months_indegoflex = df_months_usertype[df_months_usertype['user_type'] == "IndegoFlex"]
df_months_walkup = df_months_usertype[df_months_usertype['user_type'] == "Walk-up"]
df_months_onedaypass = df_months_usertype[df_months_usertype['user_type'] == "One Day Pass"]
df_months_twodaypass = df_months_usertype[df_months_usertype['user_type'] == "Two Day Pass"]

In [ ]:
# Create Bar Chart for monthly rides with Indego30
fig = plt.figure(figsize=(9,4))
plt.bar(df_months_indego30["month"], df_months_indego30["n_rides"], color = 'blue')
plt.xticks(range(1,13))
plt.rc('ytick', labelsize = 10)
plt.xlabel("month", size = 'medium')
plt.ylabel("number of rentals", size = 'medium')
plt.title("Indego 30 rentals per month", size = 'x-large')
plt.show()

In [ ]:
# Create Bar Chart for monthly rides with Walk-Up
fig = plt.figure(figsize=(9,4))
plt.bar(df_months_walkup["month"], df_months_walkup["n_rides"], color = 'red')
plt.xticks(range(1,13))
plt.rc('ytick', labelsize = 10)
plt.xlabel("month", size = 'medium')
plt.ylabel("number of rentals", size = 'medium')
plt.title("Walk-up rentals per month", size = 'x-large')
plt.show()

In [ ]:
# Create Bar Chart for monthly rides with Indego 365
fig = plt.figure(figsize=(9,4))
plt.bar(df_months_indego365["month"], df_months_indego365["n_rides"], color = 'c')
plt.xticks(range(1,13))
plt.rc('ytick', labelsize = 10)
plt.xlabel("month", size = 'medium')
plt.ylabel("number of rentals", size = 'medium')
plt.title("Indego 365 rentals per month", size = 'x-large')
plt.show()

In [ ]:
# Create Bar Chart for monthly rides with One Day Pass
fig = plt.figure(figsize=(9,4))
plt.bar(df_months_onedaypass["month"], df_months_onedaypass["n_rides"], color = 'g')
plt.xticks(range(1,13))
plt.rc('ytick', labelsize = 10)
plt.xlabel("month", size = 'medium')
plt.ylabel("number of rentals", size = 'medium')
plt.title("One Day Pass rentals per month", size = 'x-large')
plt.show()

In [ ]:
# Create Bar Chart for monthly rides with Two Day Pass
fig = plt.figure(figsize=(9,4))
plt.bar(df_months_twodaypass["month"], df_months_twodaypass["n_rides"], color = 'black')
plt.xticks(range(1,13))
plt.rc('ytick', labelsize = 10)
plt.xlabel("month", size = 'medium')
plt.ylabel("number of rentals", size = 'medium')
plt.title("Two Day Pass rentals per month", size = 'x-large')
plt.show()

In [ ]:
# Create Bar Chart for monthly rides with Indego Flex
fig = plt.figure(figsize=(9,4))
plt.bar(df_months_indegoflex["month"], df_months_indegoflex["n_rides"], color = 'orange')
plt.xticks(range(1,13))
plt.rc('ytick', labelsize = 10)
plt.xlabel("month", size = 'medium')
plt.ylabel("number of rentals", size = 'medium')
plt.title("Indego Flex rentals per month", size = 'x-large')
plt.show()

#### Analysis and Interpretation of the User Type Analysis:
The plots above all show that we the vast majority of rides (roughly 85%) are made using the Indego30 option, which is a monthly subscription. This target audience should therefore be always be a special focus. The second largest portion is attributed to spontaneous Walk-up rides which make up around 12,5%. The other are very minor compared to the monthly subscription services making up only around 2,5% of the total rides.
<br><br>Another result to note is that, the seasonality has a effect on the usage of the different payment models. Especially the One Day Pass is comparetively to other months  popular during the summer. It can be assumed that this can be accounted to the increase in tourism during those months.
<br><br>The Two Day Pass is a special case as it was seemingly only offered in the month of August in 2017. The significant lower number of rides with the One Day Pass can also be explained with the popularity of the Two Day Pass in this month. 
<br><br>The annual subscription model Indigo 365 was seeminlgy only introduced in October, but is already a popular choice even rivaling the monthly subscription model Indigo30 for the number of rides in December.
<br><br>Overall, we see a similiar monthly development for the options Indigo30, IndegoFlex and Walk-up, which were available throughout the year. Based on these results you could increase the marketing of some of the offers to either improve the usage or possibly stop some offers with low customer acceptance, like IndegoFlex.

#### Now the actual revenue analysis starts!

In [ ]:
# Prepare new dataframe for calculating the revenue
df_phil_revenue = df_philadelphia_cleaned[['end_time', 'trip_duration', 'user_type', 'weekday']].copy(deep=True)
df_phil_revenue.loc[:, 'trip_duration'] = df_phil_revenue['trip_duration'].dt.seconds / 60
df_phil_revenue.rename(columns={'trip_duration': 'trip_duration_in_minutes'}, inplace=True)
# Since we have full minutes without any remainder we can convert the trip_duration in minutes to int
df_phil_revenue['trip_duration_in_minutes'] = df_phil_revenue['trip_duration_in_minutes'].astype(int)
# The dataset from above needs to be resetted, so it doesnt need to be done here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
df_phil_revenue.reset_index(drop = True, inplace =True)
df_phil_revenue

Prices for the specific "user_type" in our dataframe were found through the help of the Wayback Machine in 2017:
- Walk-up: $4 per 30 minutes
- IndegoFlex: $4 per hour
- One Day Pass: Unlimited 30 minute trips + $2 per 30 minutes for trips over 30 minutes
- Two Day Pass: Unlimited 30 minute trips + $2 per 30 minutes for trips over 30 minutes
- Indego365: Unlimited 1 hour trips + $4 per hour for trips over 1 hour
- Indego30: Unlimited 1 hour trips + $4 per hour for trips over 1 hour

Since we were not able to find any information regarding the "Two Day Pass", we assumed that the same pricing conditions apply from the "One Day Pass" to the "Two Day Pass".

In [ ]:
# Calculation of every user_type pricing condition and saving it in a new column called "revenue_in_$"
for i in range(0, len(df_phil_revenue)):
    # We need the duration and the user_type beforehand to determine the different pricing conditions 
    duration = df_phil_revenue.at[i, 'trip_duration_in_minutes']
    price_type = df_phil_revenue.at[i, 'user_type']
    if(price_type == 'Walk-up'):
        df_phil_revenue.at[i, 'revenue_in_$'] = 4*(np.ceil((df_phil_revenue.at[i, 'trip_duration_in_minutes'])/30))
    elif(price_type == 'IndegoFlex'):
        df_phil_revenue.at[i, 'revenue_in_$'] = 4*(np.ceil((df_phil_revenue.at[i, 'trip_duration_in_minutes'])/60))
    elif(price_type == 'One Day Pass' or price_type == 'Two Day Pass'):
        # Since there are unlimited 30 minute trips, we can just give them 0$ revenue as a result
        if(duration <= 30):
            df_phil_revenue.at[i, 'revenue_in_$'] = 0
        else:
            # - 30 needs to be added to the calculation, so the first half an hour doesnt count in
            df_phil_revenue.at[i, 'revenue_in_$'] = 2*(np.ceil((df_phil_revenue.at[i, 'trip_duration_in_minutes'] - 30)/30))
    elif(price_type == 'Indego365' or price_type == 'Indego30'):
        # Since there are unlimited 60 minute trips, we can just give them 0$ revenue as a result
        if(duration <= 60):
            df_phil_revenue.at[i, 'revenue_in_$'] = 0
        else:
            # - 60 needs to be added to the calculation, so the first hour doesnt count in
            df_phil_revenue.at[i, 'revenue_in_$'] = 4*(np.ceil((df_phil_revenue.at[i, 'trip_duration_in_minutes'] - 60)/60))

# Since we dont have any values with remainder, we can also convert the revenue into an int for convenience, and drop the trip duration
df_phil_revenue['revenue_in_$'] = df_phil_revenue['revenue_in_$'].astype(int)
df_phil_revenue.drop('trip_duration_in_minutes', axis = 1, inplace = True)

In [ ]:
# Finished dataframe with all revenue calculations done
df_phil_revenue

In [ ]:
# Creating a new dataframe for easier usage later on for the plots
df_phil_revenue_plt = df_phil_revenue.copy()

# Setting the end_time as our new index to resample 
df_phil_revenue_plt.sort_values(['end_time'], inplace = True)
df_phil_revenue_plt.set_index('end_time', inplace = True)

#Finished dataframe for plotting purposes
df_phil_revenue_plt

In [ ]:
# Plotting revenue per trip based on user type
df_phil_revenue_type = df_phil_revenue_plt.copy()

# Resample based on user_type and calculate the sum of each user_type
df_phil_revenue_type = df_phil_revenue_type.groupby(['user_type']).sum().reset_index()

# Plotting
fig = plt.figure(figsize=(12,7))
plt.bar(df_phil_revenue_type['user_type'], df_phil_revenue_type['revenue_in_$'])

# Styling the plot
plt.rc('xtick', labelsize = 10)
plt.rc('ytick', labelsize = 10)
plt.xlabel('Specific user types', size = 'medium')
plt.ylabel('Revenue in $', size = 'medium')
plt.title('Revenue per trip of specific user types', size = 'x-large')
plt.show()

The most income as seen in the plot above is created by the user_type "Walk-up", since this user_type also doesnt have any kind of free one hour or half an hour usage, like "Indego30" or "Indego365". The "Two Day Pass" was only availabe in the month of August, "Indego365" since the month of October and the "One Day Pass" since the month of June, hence why these three user_types have a relatively low revenue.

In [ ]:
# Plotting the revenue for every month
sns.set(style='whitegrid', palette='colorblind', rc={'lines.linewidth': 2})

# Resampling the data to a monthly format
df_phil_revenue_plt_monthly = df_phil_revenue_plt.resample('M').sum()
df_phil_revenue_plt_monthly.index.rename('months', inplace=True)

#Plotting 
fig, ax = plt.subplots(figsize = (12,7))
plot_monthly = sns.lineplot(x = df_phil_revenue_plt_monthly.index, y = 'revenue_in_$', data = df_phil_revenue_plt_monthly)
plt.title('Revenue per trip for every month')
plt.show()

In [ ]:
# Plotting the revenue for every week
sns.set(style='whitegrid', palette='colorblind', rc={'lines.linewidth': 2})

# Resampling the data to a weekly format
df_phil_revenue_plt_weekly = df_phil_revenue_plt.resample('W').sum()
df_phil_revenue_plt_weekly.index.rename('weeks', inplace = True)

#Plotting 
fig, ax = plt.subplots(figsize = (12,7))
plot_monthly = sns.lineplot(x = df_phil_revenue_plt_weekly.index, y = 'revenue_in_$', data = df_phil_revenue_plt_weekly)
plt.title('Revenue per trip for every week')
plt.show()

In [ ]:
# Plotting the normalized revenue for every week in comparisent to the temperature
sns.set(style='whitegrid', palette='colorblind', rc={'lines.linewidth': 2})

# Preparing weather data
df_phil_revenue_weather = df_weather_2017.copy()
df_phil_revenue_weather.drop('max_temp', axis = 1, inplace = True)
df_phil_revenue_weather.drop('precip', axis = 1, inplace = True)
df_phil_revenue_weather.set_index('date_time', inplace = True)
df_phil_revenue_weather = df_phil_revenue_weather.resample('W').mean()

# Normalizing data
df_phil_revenue_plt_weekly['normalized_revenue_in_$'] = (df_phil_revenue_plt_weekly[['revenue_in_$']] - df_phil_revenue_plt_weekly[['revenue_in_$']].min(axis=0)) / (df_phil_revenue_plt_weekly[['revenue_in_$']].max(axis=0) - df_phil_revenue_plt_weekly[['revenue_in_$']].min(axis=0))
df_phil_revenue_weather['normalized_min_temp'] = (df_phil_revenue_weather[['min_temp']] - df_phil_revenue_weather[['min_temp']].min(axis=0)) / (df_phil_revenue_weather[['min_temp']].max(axis=0) - df_phil_revenue_weather[['min_temp']].min(axis=0))

#Plotting 
fig, ax = plt.subplots(figsize = (12,7))
plot_monthly = sns.lineplot(x = df_phil_revenue_plt_weekly.index, y = 'normalized_revenue_in_$', data = df_phil_revenue_plt_weekly)
plot_temp = sns.lineplot(x = df_phil_revenue_weather.index, y = 'normalized_min_temp', data = df_phil_revenue_weather)
ax.legend(['Revenue', 'Temperature'])
plt.title('Normalized revenue per trip for every week compared to normalized temperature')
plt.show()

In [ ]:
# Plotting the revenue for every day
sns.set(style='whitegrid', palette='colorblind', rc={'lines.linewidth': 2})

# Resampling the data to daily format
df_phil_revenue_plt_daily = df_phil_revenue_plt.resample('D').sum()
df_phil_revenue_plt_daily.index.rename('days', inplace = True)

#Plotting 
fig, ax = plt.subplots(figsize = (12,7))
plot_monthly = sns.lineplot(x = df_phil_revenue_plt_daily.index, y = 'revenue_in_$', data = df_phil_revenue_plt_daily)
plt.title('Revenue per trip for every day')
plt.show()

With the seasons changing and therefore rising temperature towards summer and decreasing temperature afterwards, we see a trend of the revenue first rising until August and then starting to drastically decrease. The number of tourists could also have an impact on the rise of the revenue curve in quarter 2 and the decline after quarter 3, since quarter 2 and quarter 3 show the highest numbers of tourists (as seen in the development of bike rental demand).

In [ ]:
# Plotting revenue based on weekday
df_phil_revenue_plt_weekday = df_phil_revenue_plt.copy()

# Data needs to be grouped by weekday and replaced with their corresponding weekday name
df_phil_revenue_plt_weekday = df_phil_revenue_plt_weekday.groupby(['weekday']).sum(numeric_only=True).reset_index()
df_phil_revenue_plt_weekday['weekday'].replace([0,1,2,3,4,5,6], ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], inplace=True)

# Plotting
fig = plt.figure(figsize=(12,7))
plt.bar(df_phil_revenue_plt_weekday['weekday'], df_phil_revenue_plt_weekday['revenue_in_$'])

# Styling the plot
plt.rc('xtick', labelsize = 10)
plt.rc('ytick', labelsize = 10)
plt.xlabel('weekdays', size = 'medium')
plt.ylabel('revenue in $', size = 'medium')
plt.title('Distribution of revenue per trip over the specific weekdays', size = 'x-large')
plt.show()

The weekend seems to be the time where the biggest amount of revenue is produced. Most of these people probably do not use the service very often, so they do not have any subscription model and most likely use the "Walk-up", which could explain the big jump from friday to saturday in revenue. The highest peak is at sunday. However the number of rides/rentals are higher from Monday to Friday (during the week). This leads us to the assumption, that most people who have a subscription are people who use the bike rental to get to work/university/school. 